Been a while since I last looked at this, so may miss some of the intent of the original algorithm. Rewriting here. 

# Create Approximate Schema
1. Start with b-schema graph size 0, select a triple from data graph 
    - If first step, add a random triple. All subsequent steps, select a triple connected to the existing b-schema, but not currently represented by the b-schema. 
2. Using a parameter for amount of hops (with default value of 1) create the graph around this triple. 
3. Generate a class graph for this mini graph.
4. Query the b-schema against the b-schema for this graph to see if it is already represented by the b-schema.
5. If this query fails, find which statements in the mini b-schema were not represented by the main b-schema. Add these statements to the b-schema.
6. This will create an approximate b-schema. 

# Verify schema 
1. Iterate through each triple 
2. Based on the classes and relations of the new triples, find a candidate statement in the b-schema that may represent this triple. 
3. Invert the b-schema statements to SPARQL queries with the class and relation information. 
4. Iteratively run queries with limit one with the identity of the added triple bound to the candidate statements.
    -  If one of these queries succeeds, the new triple should be added to the list of triples represented by a statement in the b-schema. 
5. If all triples are represented, end the process

In [1]:
from algo2 import * 

In [3]:
def print_bschema(b_schema_statements, save = None):
    b_schema_trpls = [stmt.pattern[0][1] for stmt in b_schema_statements]
    b_schema_graph = Graph(store = 'Oxigraph')
    for trpl in b_schema_trpls:
        b_schema_graph.add((trpl.s, trpl.p, trpl.o))
    if save is not None:
        b_schema_graph.serialize(save, format="turtle")
    b_schema_graph.print()

In [ ]:
data_graph = Graph(store = 'Oxigraph')
data_graph.parse("brick-example.ttl", format="turtle")

algo = BSchemaGenerator(data_graph)
ex_trpl = algo.all_triples[2]

query_one_hop_with_classes = f"""
{get_prefixes(data_graph)}
CONSTRUCT {{
    ?s ?p ?o .
    ?s a ?scls .
    ?o a ?cls .
}}
WHERE {{
    ?s ?p ?o . 
    ?s a ?scls .
    ?o a ?cls .
    FILTER(?s = <{ex_trpl.s}>) .
    FILTER NOT EXISTS {{
        
    }}
}}
"""

mini_bs_graph = Graph(store = 'Oxigraph')
mini_bs_graph.parse(data = data_graph.query(query_one_hop_with_classes).graph.serialize(format = 'ttl'), format = 'ttl')
mini_bs = BSchemaGenerator(mini_bs_graph)
stmts = mini_bs.generate_b_schema()


skipping subject empty string
[EXTRACT] Extracted 3428 triples
[INIT] Total triples in graph: 3428
[EXTRACT] Extracted 79 triples
[INIT] Total triples in graph: 79

STARTING B-SCHEMA GENERATION


[STEP 1] Starting with first triple: urn:example#multiple-zone-ahu_name_0 --http://www.w3.org/1999/02/22-rdf-syntax-ns#type--> https://brickschema.org/schema/Brick#AHU
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --http://www.w3.org/1999/02/22-rdf-syntax-ns#type--> http://www.w3.org/2000/01/rdf-schema#Resource

[STEP 1] Created initial statement covering 1 triples
[STEP 1] Coverage: 1/79 triples

ITERATION 1
Coverage: 1/79 triples
Current statements: 1

[CONNECT] Found connected triple: urn:example#multiple-zone-ahu_name_0 --https://brickschema.org/schema/Brick#feeds--> urn:example#vav-cooling-only_name_0_0

[STEP 2] Finding candidates for 1 new triples
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://bricksche

In [40]:
# class graph 
query_one_hop_with_classes = f"""
{get_prefixes(data_graph)}
CONSTRUCT {{
    ?s ?p ?o .
    ?s a ?scls .
    ?o a ?cls .
}}
WHERE {{
    ?s ?p ?o . 
    ?s a ?scls .
    ?o a ?cls .
    FILTER(?s = <{ex_trpl.s}>) .
}}
"""

mini_bs_graph = Graph(store = 'Oxigraph')
mini_bs_graph.parse(data = data_graph.query(query_one_hop_with_classes).graph.serialize(format = 'ttl'), format = 'ttl')
mini_bs = BSchemaGenerator(mini_bs_graph)


def create_pattern_dict(mini_bs):
    pattern_dict = {}
    for triple in mini_bs.all_triples:
        pattern = mini_bs._create_class_pattern([triple])[0][0]
        if pattern in pattern_dict.keys():
            continue
        else:
            pattern_dict[pattern] = triple
    return pattern_dict

[EXTRACT] Extracted 79 triples
[INIT] Total triples in graph: 79


In [46]:
g = Graph(store = 'Oxigraph')
pattern_dict = create_pattern_dict(mini_bs)
for pattern, triple in pattern_dict.items():
    g.add((triple.s, triple.p, triple.o))

[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --http://www.w3.org/1999/02/22-rdf-syntax-ns#type--> http://www.w3.org/2000/01/rdf-schema#Resource
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#fe

In [47]:
g.print()

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:example#multiple-zone-ahu_name_0> a brick:AHU ;
    brick:feeds <urn:example#vav-cooling-only_name_0_0> ;
    brick:hasPart <urn:example#multiple-zone-ahu_clg_coil_0>,
        <urn:example#multiple-zone-ahu_htg_coil_0>,
        <urn:example#multiple-zone-ahu_ra_damper_0>,
        <urn:example#multiple-zone-ahu_sa_fan_0> ;
    brick:hasPoint <urn:example#multiple-zone-ahu_filter_pd_0>,
        <urn:example#multiple-zone-ahu_ma_temp_0>,
        <urn:example#multiple-zone-ahu_oa_temp_0>,
        <urn:example#multiple-zone-ahu_ra_temp_0>,
        <urn:example#multiple-zone-ahu_sa_temp_0> .

<urn:example#multiple-zone-ahu_clg_coil_0> a brick:Cooling_Coil .

<urn:example#multiple-zone-ahu_filter_pd_0> a brick:Filter_Differential_Pressure_Sensor .

<urn:example#multiple-zone-ahu_htg_coil_0> a brick:Heating_Coil .

<urn:example#multiple-zone-ahu_ma_temp_0> a brick:Mixed_Air_Temperature_Sensor .

<urn:example#multiple-zone-ahu_oa_te

In [50]:
def create_class_graph(pattern_dict):
    class_graph = Graph(store = 'Oxigraph')
    for pattern, triple in pattern_dict.items():
        class_graph.add((URIRef(pattern.s), URIRef(pattern.p), URIRef(pattern.o)))
    return class_graph

In [52]:
class_graph = Graph(store = 'Oxigraph')
for triple in mini_bs.all_triples:
    pattern = mini_bs._create_class_pattern([triple])[0][0]
    class_graph.add((URIRef(pattern.s), URIRef(pattern.p), URIRef(pattern.o)))

[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --http://www.w3.org/1999/02/22-rdf-syntax-ns#type--> http://www.w3.org/2000/01/rdf-schema#Resource
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#feeds--> https://brickschema.org/schema/Brick#VAV
[PATTERN] Created pattern with 1 triples
  https://brickschema.org/schema/Brick#AHU --https://brickschema.org/schema/Brick#fe

In [22]:
mini_bs_graph.print()

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:example#multiple-zone-ahu_name_0> a brick:AHU ;
    brick:feeds <urn:example#vav-cooling-only_name_0_0>,
        <urn:example#vav-cooling-only_name_0_1>,
        <urn:example#vav-cooling-only_name_0_10>,
        <urn:example#vav-cooling-only_name_0_11>,
        <urn:example#vav-cooling-only_name_0_12>,
        <urn:example#vav-cooling-only_name_0_13>,
        <urn:example#vav-cooling-only_name_0_14>,
        <urn:example#vav-cooling-only_name_0_2>,
        <urn:example#vav-cooling-only_name_0_3>,
        <urn:example#vav-cooling-only_name_0_4>,
        <urn:example#vav-cooling-only_name_0_5>,
        <urn:example#vav-cooling-only_name_0_6>,
        <urn:example#vav-cooling-only_name_0_7>,
        <urn:example#vav-cooling-only_name_0_8>,
        <urn:example#vav-cooling-only_name_0_9>,
        <urn:example#vav-with-reheat_name_0_0>,
        <urn:example#vav-with-reheat_name_0_1>,
        <urn:example#vav-with-reheat_name_0_1

In [21]:
g = Graph(store = 'Oxigraph')
for triple in class_graph: 
    g.add((triple.s, triple.p, triple.o))
g.print()

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:example#multiple-zone-ahu_name_0> a brick:AHU ;
    brick:hasPart <urn:example#multiple-zone-ahu_clg_coil_0>,
        <urn:example#multiple-zone-ahu_htg_coil_0>,
        <urn:example#multiple-zone-ahu_ra_damper_0>,
        <urn:example#multiple-zone-ahu_sa_fan_0> ;
    brick:hasPoint <urn:example#multiple-zone-ahu_filter_pd_0>,
        <urn:example#multiple-zone-ahu_ma_temp_0>,
        <urn:example#multiple-zone-ahu_oa_temp_0>,
        <urn:example#multiple-zone-ahu_ra_temp_0>,
        <urn:example#multiple-zone-ahu_sa_temp_0> .

<urn:example#multiple-zone-ahu_clg_coil_0> a brick:Cooling_Coil .

<urn:example#multiple-zone-ahu_filter_pd_0> a brick:Filter_Differential_Pressure_Sensor .

<urn:example#multiple-zone-ahu_htg_coil_0> a brick:Heating_Coil .

<urn:example#multiple-zone-ahu_ma_temp_0> a brick:Mixed_Air_Temperature_Sensor .

<urn:example#multiple-zone-ahu_oa_temp_0> a brick:Outside_Air_Temperature_Sensor .

<urn:examp

In [ ]:
print_bschema(stmts)

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:example#multiple-zone-ahu_name_0> a brick:AHU ;
    brick:feeds <urn:example#vav-cooling-only_name_0_0> ;
    brick:hasPart <urn:example#multiple-zone-ahu_clg_coil_0>,
        <urn:example#multiple-zone-ahu_htg_coil_0>,
        <urn:example#multiple-zone-ahu_ra_damper_0>,
        <urn:example#multiple-zone-ahu_sa_fan_0> ;
    brick:hasPoint <urn:example#multiple-zone-ahu_filter_pd_0>,
        <urn:example#multiple-zone-ahu_ma_temp_0>,
        <urn:example#multiple-zone-ahu_oa_temp_0>,
        <urn:example#multiple-zone-ahu_ra_temp_0>,
        <urn:example#multiple-zone-ahu_sa_temp_0> .

<urn:example#multiple-zone-ahu_clg_coil_0> a brick:Cooling_Coil .

<urn:example#multiple-zone-ahu_filter_pd_0> a brick:Filter_Differential_Pressure_Sensor .

<urn:example#multiple-zone-ahu_htg_coil_0> a brick:Heating_Coil .

<urn:example#multiple-zone-ahu_ma_temp_0> a brick:Mixed_Air_Temperature_Sensor .

<urn:example#multiple-zone-ahu_oa_te